In [1]:
from pyspark.sql import functions as F, SparkSession, DataFrame
from pyspark.sql.types import StructField, StructType, StringType, IntegerType, LongType
from pyspark.sql.utils import AnalysisException
import json
from pathlib import Path
from delta import *

In [2]:
root_path = str(Path().resolve())
source_path = str(Path().resolve())
source_location = f'file:///{source_path}/scripts/resources'
datalake_dir = f'file:///{root_path}/datalake'
db_name = 'client_data'
storage_location = f'file:///{root_path}/{db_name}'
config_path = f'{source_path}/scripts/resources/config.json'

## Create Spark session

In [3]:
builder = SparkSession.builder.appName("ETLDemo") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config("spark.sql.warehouse.dir", datalake_dir)

In [4]:
spark = configure_spark_with_delta_pip(builder).getOrCreate()

SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/home/jinj01/hadoop/spark-3.3.3/jars/log4j-slf4j-impl-2.17.2.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/home/jinj01/hadoop/hadoop-3.3.6/share/hadoop/common/lib/slf4j-reload4j-1.7.36.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/home/jinj01/hadoop/apache-hive-3.1.2-bin/lib/log4j-slf4j-impl-2.10.0.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.apache.logging.slf4j.Log4jLoggerFactory]


2024-03-15T09:50:17,536 WARN [main] org.apache.spark.util.Utils - Your hostname, L-PF3MV5RS resolves to a loopback address: 127.0.1.1; using 172.26.122.132 instead (on interface eth0)
2024-03-15T09:50:17,544 WARN [main] org.apache.spark.util.Utils - Set SPARK_LOCAL_IP if you need to bind to another address
:: loading settings :: url = jar:file:/home/jinj01/hadoop/spark-3.3.3/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/jinj01/.ivy2/cache
The jars for the packages stored in: /home/jinj01/.ivy2/jars
io.delta#delta-core_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-4c9d5a37-1635-476a-981b-fa33bb2e5513;1.0
	confs: [default]
	found io.delta#delta-core_2.12;2.0.0 in central
	found io.delta#delta-storage;2.0.0 in central
	found org.antlr#antlr4-runtime;4.8 in central
	found org.codehaus.jackson#jackson-core-asl;1.9.13 in central
:: resolution report :: resolve 258ms :: artifacts dl 12ms
	:: modules in use:
	io.delta#delta-core_2.12;2.0.0 from central in [default]
	io.delta#delta-storage;2.0.0 from central in [default]
	org.antlr#antlr4-runtime;4.8 from central in [default]
	org.codehaus.jackson#jackson-core-asl;1.9.13 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|ev

2024-03-15T09:50:19,565 WARN [main] org.apache.hadoop.util.NativeCodeLoader - Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


2024-03-15T09:50:21,529 WARN [Thread-5] org.apache.spark.util.Utils - Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


### Query the raw data

In [5]:
client_account_df = spark.read.format("delta").load(f"{datalake_dir}/{db_name}.db/client_account")

In [6]:
client_account_df.show()

+---+--------------------+--------------------+------------------+
| id|               btc_a|                cc_t|              cc_n|
+---+--------------------+--------------------+------------------+
|  1|1wjtPamAZeGhRnZfh...|       visa-electron|  4175006996999270|
|  2|1Js9BA1rV31hJFmN2...|                 jcb|  3587679584356527|
|  3|1CoG9ciLQVQCnia5o...| diners-club-enroute|   201876885481838|
|  4|1GNvinVKGzPBVNZSc...|              switch|564182038040530730|
|  5|1DHTzZ7ypu3EzWtLB...|                 jcb|  3555559025151828|
|  6|1LWktvit3XBCJNrsj...|                 jcb|  3580083825272493|
|  7|1J71SRGqUjhqPuHaZ...|              switch|491193585665108260|
|  8|1Q5FAwgXbhRxP1uYp...|          mastercard|  5100174550682620|
|  9|1QKy8RoeWR48nrwkn...|diners-club-carte...|    30343863921001|
| 10|1NRDQBCtuDqm8Qomr...|diners-club-carte...|    30559612937267|
| 11|1HcqQ5Ys77sJm3ZJv...|                visa|     4937793997478|
| 12|1EncEr6Vd5ywk96un...|                 jcb|  3569513122126

In [7]:
client_info_df = spark.read.format("delta").load(f"{datalake_dir}/{db_name}.db/client_information")

In [8]:
client_info_df.show()

+---+----------+----------+--------------------+--------------+
| id|first_name| last_name|               email|       country|
+---+----------+----------+--------------------+--------------+
|  1|    Feliza|    Eusden|  feusden0@ameblo.jp|        France|
|  2| Priscilla|   Le Pine|plepine1@biglobe....|        France|
|  3|    Jaimie|    Sandes|jsandes2@reuters.com|        France|
|  4|      Nari|   Dolphin|ndolphin3@cbsloca...|        France|
|  5|     Garik|     Farre|gfarre4@economist...|        France|
|  6|   Kordula|   Broodes| kbroodes5@amazon.de|        France|
|  7|     Rakel|   Ingliby|    ringliby6@ft.com| United States|
|  8|      Derk| Mattielli|dmattielli7@slide...| United States|
|  9|    Karrah|   Boshard|   kboshard8@ihg.com|        France|
| 10| Friedrich|  Kreutzer|fkreutzer9@busine...|        France|
| 11|      Conn|   Claiden| cclaidena@vimeo.com|        France|
| 12|     Karel|   Crippin| kcrippinb@google.pl|        France|
| 13| Millisent|     Joint| mjointc@stat

In [14]:
id_count = client_info_df.groupBy('country').agg(F.count('id').alias('id_count')).filter(F.lower(F.col('country')).isin('united kingdom', 'netherlands'))

In [21]:
id_count.show()

+--------------+--------+
|       country|id_count|
+--------------+--------+
|United Kingdom|      38|
|   Netherlands|      62|
+--------------+--------+



In [29]:
filtered_count = id_count.select(F.sum('id_count')).collect()[0][0]

In [31]:
filtered_count

100

### Query the client data

In [9]:
client_data = spark.read.format("delta").load(storage_location)

In [10]:
client_data.show()

+-----------------+----------+---------+--------------+--------------------+--------------------+
|client_identifier|first_name|last_name|       country|     bitcoin_address|    credit_card_type|
+-----------------+----------+---------+--------------+--------------------+--------------------+
|               18|   Richard|   Drinan|United Kingdom|1ErM8yuF3ytzzxLy1...|      china-unionpay|
|               32|    Wallis|  Bamford|United Kingdom|12sxmYnPcADAXw1Yk...|             maestro|
|               33|   Saundra|  Westall|United Kingdom|1GZ7QB7GUFSWnkBHm...|          mastercard|
|               34|     Ewart|Rosengren|United Kingdom|12o8zrHx6snCPbtko...|       visa-electron|
|               36|    Daniel|Buckthorp|   Netherlands|15X53Z9B9jUNrvFpb...|diners-club-inter...|
|               62|      Boyd|   Barham|   Netherlands|16qpYVt6YAAx4JYjz...|                 jcb|
|               67|     Lorry|  Beavors|United Kingdom|12ya1ED93ApPBQRSC...|            bankcard|
|               70| 

In [18]:
client_count = client_data.count()

In [19]:
client_count

100

In [34]:
assert filtered_count == client_count